In [2]:
import os
import subprocess
from surfer import Brain, project_volume_data
from mayavi import mlab

mlab.init_notebook(backend='png')

ImportError: No module named surfer

In [ ]:
# Set environment variables
os.environ['FREESURFER_HOME'] = '/Applications/freesurfer/7.1.1'
os.environ['SUBJECTS_DIR'] = os.path.join(os.environ['FREESURFER_HOME'], 'subjects')

In [ ]:
brain = Brain(subject_id = "fsaverage5", hemi = "split", surf = "inflated", title = 'Average CBF (thr10)',
              views=['med', 'lat'], background="white")

Note: mri_file registered with following script
```
export TRANSFORM_DIR=$PWD/data/transforms 
export CBF_PNC=$PWD/data/mri/avgCBF_thr10.nii.gz
export CBF_MNI=$PWD/data/mri/reg/avgCBF_thr10_MNI.nii.gz # output path

antsApplyTransforms -i $CBF_PNC -o $PWD/data/mri/reg/avgCBF_thr10_MNI.nii.gz -t $TRANSFORM_DIR/PNC-MNI_0warp.nii.gz -t $TRANSFORM_DIR/PNC-MNI_1Affine.mat \
    -r $PWD/data/templates/MNI-2x2x2.nii.gz -d 3 -e 3 -n LanczosWindowedSinc

```

where transform

In [ ]:
mri_file = os.path.join(os.getcwd(), 'data/mri/reg/avgCBF_thr10_MNI.nii.gz')
reg_file = os.path.join(os.environ["FREESURFER_HOME"], "average/mni152.register.dat")

surf_data_lh = project_volume_data(mri_file, "lh", reg_file, smooth_fwhm=3)
surf_data_rh = project_volume_data(mri_file, "rh", reg_file, smooth_fwhm=3)


In [ ]:
brain.add_overlay(surf_data_lh, min=0, max=90, name="cbf_lh", hemi='lh')
brain.add_overlay(surf_data_rh, min=0, max=90, name="cbf_rh", hemi='rh')

In [ ]:
brain.save_image('tmp/brain_hemi-split_py2.7.png')